# 【Face Alignment其二】face alignment的3D投影算法

本文参考代码为：https://github.com/dougsouza/face-frontalization ,和博文：https://talhassner.github.io/home/publication/2014_MVAP

在[《【Face Alignment其一】用dlib进行face alignment的简单实现》](https://github.com/konatasick/machine_learning_note/blob/master/face%20alignment1.ipynb)中，讲解了Face Alignment的简单实现。但通常这种程度的alignment是不够的。目前一个主流的做法是利用人脸landmark的3D模型，将facial landmark投影至3D空间后再投影到正面上。

在源代码的readme.MD中，展示了一个例子：

![a](https://raw.githubusercontent.com/dougsouza/face-frontalization/master/example.png)

在这个源代码对应的论文中，在基础的face alignment之后增加了一个soft symmetry的步骤，用于填补脸部的缺失部分，此处算法在此暂时忽略不计。本文主要分析由query image转变为frontalized without symmetry的算法。具体论文见：[Viewing Real-World Faces in 3D](https://talhassner.github.io/home/projects/ViewFaces3D/HassnerICCV2013.pdf)和[Effective Face Frontalization in Unconstrained Images](https://talhassner.github.io/home/projects/frontalize/CVPR2015_frontalize.pdf)



## 3D投影

![a](figure/face_alignment2-1.png)

令(b)中任意一点为$p_i=(x_i,y_i)^T$，其对应的点在(c)中为$p_i^\prime=(x_i^\prime,y_i^\prime)^T$，在3D模型的(d)中为$P_i=(X_i,Y_i,Z_i)^T$。

在3D模型确认后，我们选择合适的旋转角度将其投影为一个正面的视角，用于作为参照的frontalized coordinate system，即(c)，其投影公式为：

$$p^\prime\sim C_M P$$

对于一张输入的图片，我们在得到其landmark后，计算和3D模型的投影矩阵，即：

$$p\sim C_Q P$$

对于一张输入的图片，将其转化为frontalized face的公式即为：

$$p^\prime\sim C_M C_Q^{-1} p$$



## Face frontalization
在代码中，我们首先得到输入图片的landmark

In [1]:
import frontalize
import facial_feature_detector as feature_detection
import camera_calibration as calib
import scipy.io as io
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import imutils
# load detections performed by dlib library on 3D model and Reference Image
model3D = frontalize.ThreeD_Model("model3Ddlib.mat", 'model_dlib')


# load query image
img = cv2.imread("1.jpg", 1)
org_lmarks = feature_detection.get_landmarks(img)
img=img[int(org_lmarks[0][19,1])-50:int(org_lmarks[0][8,1])+10, int(org_lmarks[0][0,0])-10:int(org_lmarks[0][16,0])+10]
plt.title('Query Image')
plt.imshow(img[:, :, ::-1])

# extract landmarks from the query image
# list containing a 2D array with points (x, y) for each face detected in the query image
lmarks = feature_detection.get_landmarks(img)
plt.figure()
plt.title('Landmarks Detected')
plt.imshow(img[:, :, ::-1])
plt.scatter(lmarks[0][:, 0], lmarks[0][:, 1])


Number of faces detected: 1
Number of faces detected: 1
